In [18]:
import pandas as pd
import numpy as np 
import glob as glob
import re
# ls

In [19]:
# #Location of Total for Histology Billing Excel File
# df = pd.read_excel("Chung CC4172 1701 PP1003 PG005815 24756.xlsx", skiprows = 3)
# df
# df['Total'].loc[40]

# #Location of Total for HTB Billing Excel File
# df_htb = pd.read_excel("+Furth CC4125 1802 PP1001 PG999999 27628 GR409796 HTB.xlsx", skiprows = 2)
# df_htb['Total'].loc[26]

# #Location of Total for IDM Billing Excel File
# df_idm = pd.read_excel("+Wellstein CC4176 1703 PP1001 PG002842 12463 IDM.xlsx", skiprows = 6)
# df_idm['Total'].loc[31]

In [20]:
file_name = [] #Create an empty list that will contain names of the file
prices = [] # Create an empty list that will contain prices
# idx = []
# count = 0

for f in glob.glob("documents/*.xlsx"):
    if f.endswith('HTB.xlsx'): #Condition that will filter the file that has a name ending with HTB
        df_htb = pd.read_excel(f, skiprows=2)
        total_price = df_htb['Total'].loc[26]
#         count += 1
        file_name.append(f)
        prices.append(total_price)
#         idx.append(count)
    elif f.endswith('IDM.xlsx'): #Condition that will filter the file that has a name ending with IDM
        df_idm = pd.read_excel(f, skiprows = 6)
        total_price = df_idm['Total'].loc[31]
#         count += 1
        file_name.append(f)
        prices.append(total_price)
#         idx.append(count)
    else:  
        df = pd.read_excel(f,skiprows = 3) #Filter the remaining files that has the name with no HTB or IDM
        total_price = df['Total'].loc[40]
#         count += 1
        file_name.append(f)
        prices.append(total_price)
#         idx.append(count)

#     print (count, '\t', f,'\t', '$', total_price)

In [21]:
worktags = pd.DataFrame({'Worktag': file_name, 'Price': prices})
worktags = worktags[['Worktag', 'Price']]
# .set_index('Index')
worktags_split = worktags['Worktag'].str.replace('(.xlsx)', '').str.split(" ", 1, expand = True)
worktags['Worktag'] = worktags_split[1]
worktags['Price'] = worktags['Price'].astype(float) 

replacements = {'Worktag' : {r'(HTB|IDM|(\b\d\d\d\d\d\b))': ''}} #Create a dictionary that will be used for dropping substrings 'HTB' and 'IDM'
worktags.replace(replacements, regex= True, inplace=True) #Drop substrings 'HTB' and 'IDM' from 'Worktag' column

# worktags = worktags.sort_values(['Price'], ascending = True)
# worktags['PI'] = worktags_split[0]
# worktags = worktags[['PI','Worktag','Price']]

worktags.Worktag = worktags.Worktag.str.split().str.join(' ')

In [22]:
invoices = pd.read_excel('documents/Invoice Details.xls', skiprows=8)
invoices = invoices[['Worktags', 'Fee']]
invoices.rename(columns={'Worktags': 'Invoice Worktags', 'Fee': 'Price'}, inplace=True)
replacements = {'Invoice Worktags' : {
                    r'(GMS Center-|Fund-|Program-|Purpose Code-|Assignee-|Grant-|Gift-|Project-|Lab Support|-|\(|\)|(\b\d{5}\b))': '',
                    r'(,)': ' '
                }
            }
invoices.replace(replacements, regex= True, inplace=True)
invoices = invoices.apply(lambda x: x.astype(str).str.upper())

sep = 'CC'
rest = invoices['Invoice Worktags'].str.split(sep, 1, expand = True)
invoices['Invoice Worktags'] = rest[1]
invoices['Invoice Worktags'] = 'CC' + invoices['Invoice Worktags'].astype(str)
invoices = invoices[:-1]
invoices['Price'] = invoices['Price'].astype(float) 

# invoices = invoices.sort_values(['Price'], ascending=True)

In [23]:
#invoices

In [24]:
frames = [worktags, invoices]
result = pd.concat(frames, axis=1)
# result

In [25]:
#result.to_excel('results.xlsx')

In [26]:
wt = worktags.set_index('Price').Worktag.str.split(expand=True).stack().to_frame('Worktag').reset_index().rename(columns={'level_1':'idx'})

In [27]:
iv = invoices.set_index('Price')['Invoice Worktags'].str.split(expand=True).stack().to_frame('Invoice Worktags').reset_index().rename(columns={'level_1':'idx'})

In [28]:
no_match_idx = wt.loc[~wt.Worktag.isin(iv['Invoice Worktags']), 'idx'].unique()

In [29]:
no_match_idx

array([], dtype=int64)

In [30]:
worktags['Matched_ID'] = ~worktags.index.isin(no_match_idx)
worktags['Matched_Value'] = worktags.Price.isin(invoices.Price)

In [31]:
# worktags

In [32]:
df3= pd.merge(worktags.assign(x=worktags['Worktag'].str.split().apply(sorted).str.join(' ')),
              invoices.assign(x=invoices['Invoice Worktags'].str.split().apply(sorted).str.join(' ')),
              on=['x', 'Price'],
              how='outer',
              indicator=True)
df3 = df3.sort_values(by='_merge')
# df3

In [33]:
# import fnmatch
# import os

# count_file = 0
# for file in os.listdir('.'):
#     if fnmatch.fnmatch(file, '*.xlsx'):
#         df = pd.read_excel(file,skiprows = 3)
#         total_price = df['Total'].loc[40]
        
#         count_file += 1
#         print (count_file, file, '|',total_price)